In [1]:
import pandas as pd
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,

)
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

## Define Model

### Params and calibration targets

In [3]:
params = {
    'treatment_duration': 0.5, # 6 months
    'screening_start_asymp': 0.,
}

bcm = get_bcm(params)

### Running Optimization

In [4]:
self_recovery_constraint = lambda params: params["smear_negative_self_recovery"] - params["smear_positive_self_recovery"]
# Create a suggested dictionary with constraints
suggested = {
    'constraint_self_recovery': self_recovery_constraint
}
# Get the number of CPU cores
num_cores = cpu_count()
# Set the number of workers for parallel optimization
num_workers = num_cores
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class,suggested=suggested, num_workers=num_cores)
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params
   

{'start_population_size': 2007771.289894175,
 'contact_rate': 0.007893909315952885,
 'rr_infection_latent': 0.23033896775422616,
 'rr_infection_recovered': 0.2159457975444834,
 'progression_multiplier': 1.1076704830849795,
 'seed_time': 1890.329466575205,
 'seed_num': 93.7762457387748,
 'seed_duration': 2.347896469799922,
 'smear_positive_death_rate': 0.35047638313161894,
 'smear_negative_death_rate': 0.03459512627691906,
 'smear_positive_self_recovery': 0.2758706768631793,
 'smear_negative_self_recovery': 0.09558474353925817,
 'screening_scaleup_shape': 0.09797199668784852,
 'screening_inflection_time': 1993.045023960771,
 'screening_end_asymp': 0.6450845569146022}

In [5]:
# mle_params = {'start_population_size': 2000286.9833452152,
#  'contact_rate': 0.006049616464180551,
#  'rr_infection_latent': 0.23006954238214838,
#  'rr_infection_recovered': 0.22466566363600388,
#  'progression_multiplier': 1.2487696283456124,
#  'seed_time': 1890.1388496601014,
#  'seed_num': 86.29293425240547,
#  'seed_duration': 1.2831367062620362,
#  'smear_positive_death_rate': 0.34122555603279875,
#  'smear_negative_death_rate': 0.030757277091233527,
#  'smear_positive_self_recovery': 0.20944123090302655,
#  'smear_negative_self_recovery': 0.10857850614221899,
#  'screening_scaleup_shape': 0.08878607354436534,
#  'screening_inflection_time': 1993.4152158420968,
#  'screening_end_asymp': 0.635603639378687}

In [6]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

### Outputs

In [7]:
plot_model_vs_actual(
    derived_df_0, targets['pop'], "total_population", "Population", "Modelled vs Data"
)

In [8]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [9]:

plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [10]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [11]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [12]:
plot_model_vs_actual(
    derived_df_0, targets['notifs'], "notification", "Notification", "Modelled vs Data"
)

In [13]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [14]:
derived_df_0['cdr'].plot()

In [15]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')
